Here we'll walk through an incremental refactor of an azure functions "app" that seperates the "core logic" from the app's concerns. 

The core logic is implemented by the `list_funcs` and `apply_func` function below. 
The `funcs` dict is meant to emulate the mapping interface of some persistent data system (e.g. file system, azure blob storage, DB, etc.).

```python
def plus_one(x):
    return x + 1

def times_two(x):
    return x + 1

funcs = {
    'plus_one': plus_one,
    'times_two': times_two,
}

def list_funcs():
    return list(funcs)

def get_func(name):
    return funcs[name]

def apply_func(*, func_input, func_name='plus_one'):
    func = get_func(func_name)
    return func(func_input)
```

In order to test each step of our refactor, we will test it the webservice like so:

In [1]:
from wip_qh.azure.test_azure_funcs_01 import test_app 

In [2]:
from wip_qh.azure.azure_funcs_01 import app

test_app(app)

In [3]:
from wip_qh.azure.azure_funcs_02 import app

test_app(app)

# Scrap: Trying to run and test azure functions locally

In [ ]:
import subprocess
import time
import requests
from contextlib import contextmanager

@contextmanager
def run_local_server(port=7071):
    """
    Launches the Azure Functions host locally in a separate process.
    Adjust the command arguments as needed for your environment.
    """
    # Launch the function host on the specified port.
    process = subprocess.Popen(
        ["func", "host", "start", "--port", str(port)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    # Allow time for the host to initialize.
    time.sleep(5)  # In a real-world scenario you might poll for readiness.
    base_url = f"http://localhost:{port}/api"
    try:
        yield base_url
    finally:
        process.terminate()
        process.wait()

def test_app_with_local_server(test_app, port=7071):
    """
    Runs the app (via a local server) in a separate process and then executes
    test_app against the live server endpoints.

    Parameters:
      app: The FunctionApp instance (not used directly in this implementation,
           but kept for signature consistency).
      test_app: A function that accepts a base_url string and performs tests
                by sending HTTP requests to the running server.
      port: Port on which to run the local server.
    """
    with run_local_server(port) as base_url:
        # The test_app function should be written to use the provided base_url.
        # For example, it might do:
        #    response = requests.get(base_url + "/list_funcs")
        #    ... perform assertions on response ...
        test_app(base_url)

def app_tester(base_url):
    # Test the list_funcs route.
    response = requests.get(f"{base_url}/list_funcs")
    assert response.status_code == 200, "list_funcs endpoint failed"
    data = response.json()
    assert "plus_one" in data, "Missing plus_one in list"
    assert "times_two" in data, "Missing times_two in list"

    # Test the apply_func route with plus_one.
    response = requests.get(f"{base_url}/apply_func", params={"arg": "3", "func_name": "plus_one"})
    assert response.status_code == 200, "apply_func endpoint (plus_one) failed"
    data = response.json()
    assert data.get("result") == 4, "Incorrect result for plus_one"

test_app_with_local_server(app_tester)

ConnectionError: HTTPConnectionPool(host='localhost', port=7071): Max retries exceeded with url: /api/list_funcs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108f1ce80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [28]:

def app_tester(base_url):
    # Test the list_funcs route.
    response = requests.get(f"{base_url}/list_funcs")
    assert response.status_code == 200, "list_funcs endpoint failed"
    data = response.json()
    assert "plus_one" in data, "Missing plus_one in list"
    assert "times_two" in data, "Missing times_two in list"

    # Test the apply_func route with plus_one.
    response = requests.get(f"{base_url}/apply_func", params={"arg": "3", "func_name": "plus_one"})
    assert response.status_code == 200, "apply_func endpoint (plus_one) failed"
    data = response.json()
    assert data.get("result") == 4, "Incorrect result for plus_one"

app_tester("http://localhost:7071/api")

AssertionError: list_funcs endpoint failed

In [34]:
import requests

r = requests.get('http://localhost:7071/api/list_funcs')
r


<Response [500]>

# AI contexts

Here we'll walk through an incremental refactor of an azure functions "app" that seperates the "core logic" from the app's concerns. 

The core logic is implemented by the `list_funcs` and `apply_func` function below. 
The `funcs` dict is meant to emulate the mapping interface of some persistent data system (e.g. file system, azure blob storage, DB, etc.).

```python
# wip_qh/azure/core_logic.py

def plus_one(x):
    return x + 1

def times_two(x):
    return x + 1

funcs = {
    'plus_one': plus_one,
    'times_two': times_two,
}

def list_funcs():
    return list(funcs)

def get_func(name):
    return funcs[name]

def apply_func(*, func_input, func_name='plus_one'):
    func = get_func(func_name)
    return func(func_input)
```

The following is what a straightforward implementation of the web service 
(or microservice), serving the `list_funcs` and `apply_func` might look like.
The way you'd see a "Hello, World!" in a Azure Functions tutorial.

```python
# wip_qh.azure.azure_funcs_01.py
"""
The following is what a straightforward implementation of the web service 
(or microservice), serving the `list_funcs` and `apply_func` might look like.
The way you'd see a "Hello, World!" in a Azure Functions tutorial.
"""

from wip_qh.azure.core_logic import (
    list_funcs as _list_funcs,
    apply_func as _apply_func, 
)

import json
import azure.functions as af

# Create the Function App with anonymous access.
app = af.FunctionApp(http_auth_level=af.AuthLevel.ANONYMOUS)

# --- Azure Functions route definitions ---

@app.route(route="list_funcs", methods=["GET", "POST"])
def list_funcs(req: af.HttpRequest) -> af.HttpResponse:
    names = _list_funcs()
    return af.HttpResponse(
        json.dumps(names), status_code=200, mimetype="application/json"
    )


@app.route(route="apply_func", methods=["GET", "POST"])
def apply_func(req: af.HttpRequest) -> af.HttpResponse:
    # Attempt to extract 'arg' either from query parameters or JSON body.
    try:
        arg = req.params.get("arg")
        if not arg:
            req_body = req.get_json()
            arg = req_body.get("arg")
    except ValueError:
        return af.HttpResponse("Invalid request body", status_code=400)

    # Extract the function name (defaulting to 'plus_one').
    func_name = req.params.get("func_name")
    if not func_name:
        try:
            req_body = req.get_json()
            func_name = req_body.get("func_name")
        except Exception:
            raise KeyError("Function name not found")

    # Convert 'arg' to an integer.
    try:
        arg = int(arg)
    except Exception:
        return af.HttpResponse("Invalid arg value", status_code=400)

    try:
        result = _apply_func(arg=arg, func_name=func_name)
        return af.HttpResponse(
            json.dumps({"result": result}), status_code=200, mimetype="application/json"
        )
    except KeyError:
        return af.HttpResponse("Function not found", status_code=404)

```


Observe that there is a whole lot of violations of the SoC and DRY principles here. 
See what all that code is actually doing: Just 
- extracting the arguments of the target function from the req object, 
- calling the target function with those arguments, and then 
- either returning the output, wrapped in some context appropriate stuff (json serializing and returning an HttpResponse object) 
- or if there's an error, returning a HttpResponse that contains error information

This is `ingress` and `egress` wrapping (along with special error handling) where the `ingress` function takes care of processing the input (a `HttpRequest` instance) to make it a valid input for the target function, and handling errors and outputs are cast with `egress`. 

Refactor the code so we can get a reusable mechanism to make such routes. 



Observe that there is a whole lot of violations of the SoC and DRY principles here. 
Really, you should be able to have an `add_route` function that would be used like this:

```python
from wip_qh.azure.core_logic import list_funcs, apply_func


import json
import azure.functions as af

# Create the Function App with anonymous access.
app = af.FunctionApp(http_auth_level=af.AuthLevel.ANONYMOUS)

add_route(list_funcs)
add_route(apply_func)
```

that would have exactly the same effect. 

`add_route` would look something like this:



In [ ]:
The following solution demonstrates one approach to separating core logic from HTTP concerns by abstracting the “ingress” (request parameter extraction and conversion) and “egress” (wrapping output in an HTTP response) logic into a reusable decorator. This mechanism centralizes common tasks such as:
	•	Extracting expected parameters from both the query string and the JSON body.
	•	Converting parameters (e.g., from strings to integers) using provided conversion functions.
	•	Mapping HTTP parameter names to the names expected by the core functions.
	•	Handling errors (for missing or invalid parameters) uniformly.
	•	Wrapping successful outputs in a JSON-formatted HTTP response.

The reusable mechanism can be implemented in a dedicated module (e.g. wip_qh/azure/route_adapter.py). One possible implementation is shown below:

# wip_qh/azure/route_adapter.py

import json
import azure.functions as af
from functools import wraps

def route_handler(expected_params=None, param_map=None):
    """
    A decorator to wrap a core function as an Azure Functions route handler.
    
    Parameters:
      - expected_params: A dict mapping HTTP parameter names to conversion functions.
        Each conversion function is applied to the raw parameter value (from the query string
        or JSON body) to produce the proper type. If a parameter is missing, a 400 response is returned.
      - param_map: A dict mapping HTTP parameter names to the corresponding names expected
        by the core function. If not provided, the HTTP parameter names are used directly.
    
    The decorated function should accept the core parameters (after mapping) as keyword arguments.
    """
    expected_params = expected_params or {}
    param_map = param_map or {}
    
    def decorator(core_func):
        @wraps(core_func)
        def wrapper(req: af.HttpRequest):
            try:
                # Attempt to parse a JSON body (if available); fall back to an empty dict otherwise.
                try:
                    body = req.get_json()
                except Exception:
                    body = {}
                
                # Extract expected parameters from the query string or JSON body.
                gathered = {}
                for http_param, converter in expected_params.items():
                    value = req.params.get(http_param)
                    if value is None:
                        value = body.get(http_param)
                    if value is None:
                        return af.HttpResponse(f"Missing parameter: {http_param}", status_code=400)
                    if converter is not None:
                        try:
                            value = converter(value)
                        except Exception as e:
                            return af.HttpResponse(f"Invalid value for parameter '{http_param}': {e}", status_code=400)
                    gathered[http_param] = value
                
                # Map HTTP parameter names to the core function's parameter names.
                core_args = {}
                for key, value in gathered.items():
                    core_key = param_map.get(key, key)
                    core_args[core_key] = value
                
                # Call the core function with the processed arguments.
                result = core_func(**core_args)
                return af.HttpResponse(
                    json.dumps({"result": result}),
                    status_code=200,
                    mimetype="application/json"
                )
            except KeyError as e:
                # Specific errors (e.g. missing functions) can be mapped to a 404 response.
                return af.HttpResponse(str(e), status_code=404)
            except Exception as e:
                return af.HttpResponse(str(e), status_code=400)
        return wrapper
    return decorator

Using this decorator, the Azure Functions routes become much more concise. For example, consider a core logic module defined in wip_qh/azure/core_logic.py:

# wip_qh/azure/core_logic.py

def plus_one(x):
    return x + 1

def times_two(x):
    return x * 2  # Corrected to actually multiply by two.

funcs = {
    'plus_one': plus_one,
    'times_two': times_two,
}

def list_funcs():
    return list(funcs)

def get_func(name):
    return funcs[name]

def apply_func(*, func_input, func_name='plus_one'):
    func = get_func(func_name)
    return func(func_input)

The refactored Azure Functions routes can now be defined as follows:

# wip_qh/azure/azure_funcs_02.py

import azure.functions as af
from wip_qh.azure.core_logic import list_funcs as _list_funcs, apply_func as _apply_func
from wip_qh.azure.route_adapter import route_handler

app = af.FunctionApp(http_auth_level=af.AuthLevel.ANONYMOUS)

# Route for listing available functions.
@app.route(route="list_funcs", methods=["GET", "POST"])
@route_handler()  # No parameters required.
def list_funcs():
    return _list_funcs()

# Route for applying a function to an input.
@app.route(route="apply_func", methods=["GET", "POST"])
@route_handler(
    expected_params={
        'arg': int,  # Convert the 'arg' parameter to an integer.
        'func_name': lambda x: x or 'plus_one'  # Use 'plus_one' as a default if no function is specified.
    },
    param_map={
        'arg': 'func_input'  # Map the HTTP parameter 'arg' to the core function's 'func_input'.
    }
)
def apply_func(func_input, func_name):
    return _apply_func(func_input=func_input, func_name=func_name)

In this refactored design:
	•	The route_handler decorator encapsulates the repetitive tasks of extracting parameters, validating them, and formatting the response.
	•	The core functions in core_logic.py remain entirely free of any HTTP-specific code.
	•	The Azure Functions routes are now streamlined, delegating the common logic to the decorator. This not only promotes the separation of concerns (SoC) but also adheres to the DRY (Don’t Repeat Yourself) principle, making future routes simpler to implement and maintain.

This pattern can be extended or customized further (for example, by adding support for default values, logging, or more nuanced error handling) as the application’s complexity grows.